In [1]:
import investpy
from datetime import datetime, timedelta,timezone
import pandas as pd
import pytz

In [2]:
def get_economic_calendar(start_date, end_date):
    try:
        # Retrieve the economic calendar
        calendar = investpy.economic_calendar(
            from_date=start_date.strftime('%d/%m/%Y'),
            to_date=end_date.strftime('%d/%m/%Y')
        )
        
        # Convert all datetime columns to UTC+4
        # if 'date' in calendar.columns:
        #     # Adjust the 'date' column to UTC+4 by adding 4 hours
        #     calendar['date']=pd.to_datetime(calendar['date'])
        #     calendar['date'] = calendar['date'] + timedelta(hours=4)
        
        return calendar
    except Exception as e:
        print(f"Error occurred: {e}")
        return None

def get_time_in_azerbaijan():
    """Get the current time in Azerbaijan timezone."""
    azerbaijan_tz = pytz.timezone('Asia/Baku')
    return datetime.now(azerbaijan_tz)

start_date = datetime.now() 
end_date = datetime.now()+timedelta(days=4)

economic_events = get_economic_calendar(start_date, end_date)

df=pd.DataFrame(economic_events)
countries=['united states','united kingdom','euro zone','germany','canada','japan']
df=df[df['zone'].isin(countries)]
df=df[((df['zone']=='united states')  & (df['importance']!='low')) | (df['importance']=='high')]
df['date']=pd.to_datetime(df['date'])
try:
    df['datetime'] = pd.to_datetime(df['date'].dt.strftime('%Y-%m-%d') + ' ' + df['time'])
except Exception as e:
    print(f"Error combining date and time: {e}")
    
server_time = datetime.now()
azerbaijan_time = get_time_in_azerbaijan()

azerbaijan_time_naive = azerbaijan_time.replace(tzinfo=None)

diff_hours = (azerbaijan_time_naive - server_time).total_seconds() / 3600
if diff_hours > 0:
    df['datetime'] = df['datetime'] - timedelta(hours=diff_hours)
elif diff_hours < 0:
    df['datetime'] = df['datetime'] + timedelta(hours=abs(diff_hours))
else:
    pass
df = df[df['time'] != 'All Day']

if 'datetime' in df.columns and not df['datetime'].isnull().all():
    df['date'] = df['datetime'].dt.date
    df['time'] = df['datetime'].dt.time

df=df[['date', 'time', 'currency', 'importance', 'event', 'forecast', 'previous']]
df=df.sort_values(by=['importance','date'])

In [3]:
df

,date,time,currency,importance,event,forecast,previous
8,2024-09-12,03:50:00,JPY,high,GDP (QoQ) (Q3),0.2%,0.2%
104,2024-10-12,11:00:00,EUR,high,German CPI (MoM) (Nov),None,-0.2%
180,2024-11-12,17:30:00,USD,high,Core CPI (MoM) (Nov),None,0.3%
182,2024-11-12,17:30:00,USD,high,CPI (MoM) (Nov),None,0.2%
183,2024-11-12,17:30:00,USD,high,CPI (YoY) (Nov),None,2.6%
188,2024-11-12,18:45:00,CAD,high,BoC Interest Rate Decision,None,3.75%
189,2024-11-12,19:30:00,USD,high,Crude Oil Inventories,None,-5.073M
211,2024-11-12,22:00:00,USD,high,10-Year Note Auction,None,4.347%
280,2024-12-12,17:15:00,EUR,high,Deposit Facility Rate (Dec),None,3.25%
281,2024-12-12,17:15:00,EUR,high,ECB Interest Rate Decision (Dec),None,3.40%


In [4]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def send_email(receiver_email, subject, html_body):
    sender_email = "turalaliyev.142002@gmail.com"
    password = ""  "lidi ylth vbrb anpi"

    # Set up the email content
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = receiver_email
    message['Subject'] = subject  

    # Attach the HTML body
    message.attach(MIMEText(html_body, 'html'))

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message.as_string())
        print("Email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")
    finally:
        server.quit()

# Convert DataFrame to HTML with inline CSS for styling
html_body = df.to_html(index=False, justify='center', border=0)
html_body = f'''
<html>
<head>
<style>
    table {{
        width: 100%;
        border-collapse: separate;
        border-spacing: 0;
        font-size: 14px;
    }}
    th, td {{
        padding: 8px;
        text-align: left;
        white-space: nowrap;
    }}
    th {{
        background-color: #4CAF50;
        color: white;
    }}
    tr {{
        border-bottom: 1px solid #ddd;
    }}
    tr:last-child {{
        border-bottom: none;
    }}
</style>
</head>
<body>
    <h3>Economic Data Releases</h3>
    {html_body}
</body>
</html>
'''

# Send the email with the HTML table
subject = "Economic Data Releases"
receiver_email = "aliyevtural142002@gmail.com"  
send_email(receiver_email, subject, html_body)


Email sent successfully!
